# Calibration
The purpose of this notebook is to calibrate our data based on peaks, following Matt's work [here (login required)](https://zzz.physics.umn.edu/!cdms/cdms/k100/run_summary/run_76/electron-equivalent_energy_calibration).

In [1]:
#Imports
import sys
import numpy as np              #Will be used for binning
import pandas as pd             #Will be used for data structuring
import lmfit as lf              #Fitting
import matplotlib.pyplot as plt #Basic plotting library
from scipy.stats import kstest as ks
sys.path.append('../python')
from R76Tools import *          #Tools written by me for, e.x., importing our files

#Constants
frittspath = "/data/chocula/fritts/data/k100proc/midasrq/byseries/"
neogpath = "/data/chocula/neog/rq/"
baselinecorrections = pd.read_csv("../baseline_correction/baselinecorrectionvalues.csv")
gaus_mod = lf.Model(gaus)
gaus_params = gaus_mod.make_params()
#gaus_params.add('A',min=0,value=300)
gaus_params.add('sigma',min=0,value=3,max=200)
gaus_params.add('mu',min=100,max=300,value=150)
cal_mod = lf.Model(E_cal)
cal_params = cal_mod.make_params(a=300,c=13)

/data/chocula/harrisk/R76Analysis/c-to-py-translations/../python/R76Tools.py:8: UserWarning: Module `pysftp` was missing. Online functions for use of data via SSH will be unavailable.
  warnings.warn("Module `pysftp` was missing. Online functions for use of data via SSH will be unavailable.")#,warnings.ImportWarning)


In [2]:
#User-set Variables
#A little caching trick
#PuBe, Full Shielding, -81V, NaI Trigger 7.6-11.2 MeV, skip crashed or otherwise bad entries. 57 hours of data.
ser_new = ["07220916_2059","07220916_2200","07220917_1039","07220917_1225","07220917_2125","07220917_2155","07220919_1723",
    "07220919_2012","07220920_0925","07220920_1007","07220920_2052","07220920_2138","07220921_0915","07220921_1034",
    "07220921_1541","07220921_1631","07220921_2055"]
datapath_new = frittspath
%store -r ser datapath
if not (ser_new == ser and datapath_new == datapath):
    print("Loading new files; building z.")
    calibrationaliases.append("cofintl"); print(calibrationaliases)
    _,z = makechain_list(ser,path=datapath,filters=[fittingfilters,"PTOFamps0"],aliases=calibrationaliases)
    %store ser datapath z
else:
    %store -r z
#lab = ["0V","-4V","-21V","-65V","-84V","-84V"]

#Derived variables

In [3]:
# Fix up the names
for i,x in enumerate(z):
    for j in range(len(baselinecorrections)): # Pull m and b values from the csv
        if baselinecorrections.series[j]==ser[i]:
            m=baselinecorrections.m[j]
            b=baselinecorrections.b[j]
    x["cgoodwalk"] = (x["PCWKr20"]>0.25e-3)&(x["PCWKr20"]<0.5e-3)&(x["PDWKr20"]>0.25e-3)&(x["PDWKr20"]<0.5e-3)&\
        (x["PEWKr20"]>0.25e-3)&(x["PEWKr20"]<0.5e-3)
    x["pt_keV_bscorr"] = x["pt_keV"]/(1+m*x["BSel"]/b)
    x["pt0_keV_bscorr"] = x["pt0_keV"]/(1+m*x["BSel"]/b)

In [4]:
z_tot = pd.concat([x for x in z],axis=0)
print(z_tot)

        EventCategory      PAOFamps   PAWKf20   PAWKf40   PAWKr20   PAWKr40  \
0                 0.0  6.312980e-07  0.000000  0.002072  0.000399  0.000404   
1                 0.0  1.838609e-07  0.000643  0.000558  0.000382  0.000387   
2                 0.0  6.345158e-07  0.001244  0.000970  0.000401  0.000406   
3                 0.0  5.879847e-07  0.001054  0.000786  0.000404  0.000413   
4                 0.0  6.714255e-07  0.000000  0.000000  0.000400  0.000404   
...               ...           ...       ...       ...       ...       ...   
446845            0.0  2.403316e-09  0.001813  0.001614  0.000297  0.000372   
446846            0.0  4.574314e-07  0.002057  0.001337  0.000397  0.000402   
446847            0.0  4.411289e-07  0.002297  0.001480  0.000396  0.000401   
446848            0.0  1.477483e-07  0.002961  0.002572  0.001936  0.001940   
446849            1.0  1.294976e-09  0.003134  0.002790  0.001187  0.001372   

               PAbs  PAbspost      PBOFamps   PBWKf

In [ ]:
i=2
lolim=0
hilim=10
bins = np.linspace(lolim,hilim,200)
h4ptdat = z_tot["pt_keV_bscorr"][~z_tot["crand"] & z_tot["cbs"] & z_tot["cgoodwalk"] & z[i]["cofintl"]]
h4pt = plt.hist(h4ptdat,bins=bins,histtype="step",label="goodwalk")
plt.hist(z_tot["pt_keV_bscorr"][~z_tot["crand"] & z_tot["cbs"]],bins=bins,histtype="step")
plt.legend()
plt.show()

In [ ]:
for idx,y in enumerate(z):
    plt.hist(y['pt_keV_bscorr'][~y['crand'] & y['cbs']],bins=bins,histtype='step',label=idx)
plt.hist(z_tot['pt_keV_bscorr'][~z_tot['crand'] & z_tot['cbs']],bins=bins,histtype='step',label='total')
plt.legend()
plt.xlim(lolim,hilim)
plt.show()

In [ ]:
weights = 1/np.sqrt(h4pt[0]) #weights are 1/uncertainty which are Poisson so 1/sqrt(data)
print(np.isnan(weights).any(),np.isnan(bins).any(),np.isnan(h4pt[0]).any())

In [ ]:
#Initial plotting
fit1lo,fit1hi=155,175
fit2lo,fit2hi=175,190

plt.hist(h4ptdat,bins=bins,histtype="step")

#Lmfit setup

bins = (h4pt[1][:-1] + h4pt[1][1:])/2 #restructure to align for calculations
#fit 1st region
region1 = (bins >= fit1lo) & (bins <= fit1hi)
highestregion1 = h4pt[0][region1]==max(h4pt[0][region1])
gaus_params.add('mu',min=fit1lo,max=fit1hi,value=bins[region1][highestregion1][0])
fit1 = gaus_mod.fit(h4pt[0][region1],gaus_params,x=bins[region1],weights=weights[region1])
#fit 2nd region
region2 = (bins >= fit2lo) & (bins <= fit2hi)
highestregion2 = h4pt[0][region2]==max(h4pt[0][region2])
gaus_params.add('mu',min=fit2lo,max=fit2hi,value=bins[region2][highestregion2][0])
fit2 = gaus_mod.fit(h4pt[0][region2],gaus_params,x=bins[region2],weights=weights[region2])

#Plot fits
plt.plot(bins[region1],fit1.best_fit)
plt.plot(bins[region2],fit2.best_fit)
plt.xlim(lolim,hilim)
plt.suptitle("pt_keV_bscorr, series "+ser[i])
plt.title("Cuts: no random events, Baseline")
plt.xlabel('keV'); plt.ylabel('Frequency')
plt.show()
x = [fit1.best_values['mu'],fit2.best_values['mu']]
y = y_calib
c = y[0]*y[1]*(x[1]-x[0])/(x[0]*y[1]-x[1]*y[0])
a = x[0]*(c/y[0] + 1)
print(ser[i],"a:",round(a),"c:",round(c))

In [ ]:
fit1

In [ ]:
fit2

In [ ]:
#Calculate our calibration
x = [fit1.best_values['mu'],fit2.best_values['mu']]
y = y_calib
c = y[0]*y[1]*(x[1]-x[0])/(x[0]*y[1]-x[1]*y[0])
a = x[0]*(c/y[0] + 1)
#Apply the result
z[4]["pt_keVee"] = c*z[4]["pt_keV_bscorr"]/(a-z[4]["pt_keV_bscorr"])
z[4]["pt0_keVee"] = c*z[4]["pt0_keV_bscorr"]/(a-z[4]["pt0_keV_bscorr"])
#And plot
h4keVee = plt.hist(z[4]["pt_keVee"][~z[4]["crand"] & z[4]["cbs"]],bins=np.linspace(10,30,200))
plt.xlim(10,30,200)
plt.show()

In [ ]:
#Diagnostics
g_NTL_empirical = a/c     #Calculated gain from calibration
g_NTL_expected = 1+84/3.8 #1+HV/mean energy to produce one pair for Si
print("Emperical:",g_NTL_empirical)
print("Expected:",g_NTL_expected)

h4n = plt.hist(z[4]["pt0_keVee"][~z[4]["crand"] & z[4]["cbs"]],bins=np.linspace(-0.1,0.1,200))
gaus_params.add('mu',value=0)
bins = (h4n[1][:-1]+h4n[1][1:])/2
fit_NTL = gaus_mod.fit(h4n[0],gaus_params,x=bins)#,weights=1/np.sqrt(h4n[0]))
plt.plot(bins,fit_NTL.best_fit)
plt.show()
fit_NTL

In [ ]:
print(a,c)

# 241 Am Calibration

In [ ]:
#User-set Variables
#A little caching trick
#PuBe, Full Shielding, -81V, NaI Trigger 7.6-11.2 MeV, skip crashed or otherwise bad entries. 57 hours of data.
am_new = ["07220822_1828","07220826_1219","07220826_1536","07220826_2007","07220827_1153","07220830_1724"]
datapath_am_new = frittspath
%store -r am datapath_am
if not (am_new == am and datapath_am_new == datapath_am):
    print("Loading new files; building z.")
    calibrationaliases.append("cofintl"); print(calibrationaliases)
    am = am_new; datapath_am = datapath_am_new
    _,z_am = makechain_list(am,path=datapath_am,filters=[fittingfilters,"PTOFamps0"],aliases=calibrationaliases)
    %store am datapath_am z_am
else:
    %store -r z_am
#lab = ["0V","-4V","-21V","-65V","-84V","-84V"]

#Derived variables

In [ ]:
# Fix up the names
for i,x in enumerate(z_am):
    for j in range(len(baselinecorrections)): # Pull m and b values from the csv
        if baselinecorrections.series[j]==ser[i]:
            m=baselinecorrections.m[j]
            b=baselinecorrections.b[j]
    x["cgoodwalk"] = (x["PCWKr20"]>0.25e-3)&(x["PCWKr20"]<0.5e-3)&(x["PDWKr20"]>0.25e-3)&(x["PDWKr20"]<0.5e-3)&\
        (x["PEWKr20"]>0.25e-3)&(x["PEWKr20"]<0.5e-3)
    m=-0.0446154; b=154
    #x["pt_keV"]=7.738820e+07*x["PTOFamps"]+1.653756e+13*x["PTOFamps"]**2
    x["pt_keV_bscorr"] = x["pt_keV"]/(1+m*x["BSel"]/b)
    x["pt0_keV_bscorr"] = x["pt0_keV"]/(1+m*x["BSel"]/b)
    x["cam"] = x['PFOFamps']/x['PTOFamps']>0.21
    x["cphi1"] = (x['phidel']>5)&(x['phidel']<20)
    x["cbs"] = x["BSel"]<1250

#z_am_tot = pd.concat([x for x in z_am],axis=0)

In [ ]:
i=4
lolim=100
hilim=300
bins = np.linspace(lolim,hilim,200)
h4ptdat = z_am[i]["pt_keV_bscorr"][~z_am[i]["crand"] & z_am[i]["cbs"] #& z_am[i]["cgoodwalk"] 
    #& z_am[i]["cofintl"] 
    #& z_am[i]["cam"] 
    #& z_am[i]["cphi1"]
]
h4pt = plt.hist(h4ptdat,bins=bins,histtype="step",label="am")
#plt.hist(z_am[i]["pt_keV"][~z_am[i]["crand"] & z_am[i]["cbs"]],histtype='step',bins=bins)
#plt.hist(z_am[i]["pt_keV_bscorr"][~z_am[i]["crand"] & z_am[i]["cbs"]],bins=bins,histtype="step")
#plt.legend()
plt.xlim(lolim,hilim);plt.ylim(0,1.1*max(h4pt[0]))
plt.show()

weights = 1/np.sqrt(h4pt[0]) #weights are 1/uncertainty which are Poisson so 1/sqrt(data)

In [ ]:
#Initial plotting
fit1lo,fit1hi=138,158
fit2lo,fit2hi=158,172

plt.hist(h4ptdat,bins=bins,histtype="step")

#Lmfit setup

bins = (h4pt[1][:-1] + h4pt[1][1:])/2 #restructure to align for calculations
#fit 1st region
region1 = (bins >= fit1lo) & (bins <= fit1hi)
highestregion1 = h4pt[0][region1]==max(h4pt[0][region1])
gaus_params.add('mu',min=fit1lo,max=fit1hi,value=bins[region1][highestregion1][0])
gaus_params.add('A',value=max(h4pt[0][region1]))
gaus_params.add('sigma',value=5)
fit1 = gaus_mod.fit(h4pt[0][region1],gaus_params,x=bins[region1],weights=weights[region1])
#fit 2nd region
region2 = (bins >= fit2lo) & (bins <= fit2hi)
highestregion2 = h4pt[0][region2]==max(h4pt[0][region2])
gaus_params.add('mu',min=fit2lo,max=fit2hi,value=bins[region2][highestregion2][0])
gaus_params.add('A',value=max(h4pt[0][region2]))
gaus_params.add('sigma',value=5)
fit2 = gaus_mod.fit(h4pt[0][region2],gaus_params,x=bins[region2],weights=weights[region2])

#Plot fits
plt.plot(bins[region1],fit1.best_fit,color='red')
plt.plot(bins[region2],fit2.best_fit,color='red')
plt.xlim(lolim,200)
#plt.suptitle("pt_keV_bscorr, series "+ser[i])
plt.title("Cuts: Non-Randoms, Baseline, Pileup, Phi")
plt.xlabel('keV'); plt.ylabel('Frequency')
plt.show()
x = [fit1.best_values['mu'],fit2.best_values['mu']]
y = y_calib
c = y[0]*y[1]*(x[1]-x[0])/(x[0]*y[1]-x[1]*y[0])
a = x[0]*(c/y[0] + 1)
print(ser[i],"a:",round(a),"c:",round(c))

In [ ]:
fit1

In [ ]:
fit2

# Noise Width

In [ ]:
#Debug
plt.figure(figsize=(16,9))

z_tot["pt_keVee_root"] = 13.1849*z_tot["pt_keV_bscorr"]/(297.667-z_tot["pt_keV_bscorr"])
z_tot["pt0_keVee_root"] = 13.1849*z_tot["pt0_keV_bscorr"]/(297.667-z_tot["pt0_keV_bscorr"])

h4n = plt.hist(z_tot["pt0_keVee_root"][z_tot["crand"] & z_tot["cbs"]],bins=np.linspace(-0.1,0.1,200),alpha=0.6)
gaus_params.add('mu',value=0,min=-0.1,max=0.1)
gaus_params.add('sigma',value=0.025,min=0) #Guess visually
gaus_params.add('A',value=max(h4n[0]),min=0.9*max(h4n[0]))
bins = (h4n[1][:-1]+h4n[1][1:])/2
fit_NTL = gaus_mod.fit(h4n[0],gaus_params,x=bins,weights=1/np.sqrt(h4n[0]))#,weights=1/np.sqrt(h4n[0]))

def cauchy(x,A=1,gamma=1):
    return A*(gamma/(x**2+gamma**2))
cauchy_mod = lf.Model(cauchy)
cauchy_params = cauchy_mod.make_params()
cauchy_params.add('A',value=max(h4n[0]))
fit_NTL_cauchy = cauchy_mod.fit(h4n[0],cauchy_params,x=bins,weights=1/np.sqrt(h4n[0]))

def t(x,A=1,v=1):
    v = int(v)
    return A*(1+x**2/v)**(-(v+1)/2)
t_mod = lf.Model(t)
t_params = t_mod.make_params()
t_params.add('A',value=max(h4n[0]))
t_params.add('v',value=5,min=1)
fit_NTL_t = t_mod.fit(h4n[0],t_params,x=bins,weights=1/np.sqrt(h4n[0]))

plt.plot(bins,fit_NTL.best_fit,label='Gaussian')
plt.plot(bins,fit_NTL_cauchy.best_fit,label='Cauchy')
#plt.plot(bins,fit_NTL_t.best_fit,label='t')
#plt.plot(bins,t(bins,max(h4n[0]),1e10))

plt.text(-0.105,18000,"$\sigma$: 9.6 eV$_{ee}$\n Red $\chi^2$: 146",bbox=dict(color='orange',alpha=0.6),fontsize=20)
plt.text(-0.105,15000,"$\gamma$: 6.3 eV$_{ee}$\n Red $\chi^2$: 162",bbox=dict(color='g',alpha=0.6),fontsize=20)

plt.legend(fontsize=20)
plt.xlabel('Noise Level (keV$_{ee}$)',fontsize=20)
plt.ylabel('Frequency',fontsize=20)
plt.tick_params(axis='both',which='major',labelsize=20)
plt.show()
fit_NTL

In [ ]:
fit_NTL_cauchy

In [ ]:
fit_NTL_t